In [4]:
import import_ipynb

In [5]:
from ParsingCSV import parseCSV

all_contributions, timestamps, all_folders = parseCSV('contributions.csv')

importing Jupyter notebook from ParsingCSV.ipynb


In [6]:
from ManipulationMethods import only_one_folder

# Cluster within just one folder
all_nodes = list(all_contributions.values())

folder = "hw4"

# Returns a list of all nodes where the question is contained in folder
folder_contributions = only_one_folder(folder, all_nodes)

importing Jupyter notebook from ManipulationMethods.ipynb


In [15]:
from ClusteringMethods import kmeans_cluster
from ManipulationMethods import create_cluster_dicts
from ManipulationMethods import short_question_to_node_dict

# Should put optimal k here
k = 5

# Creates a dictionary that links the first 100 characters of the shortened question to the raw node
# so that it can be retrieved later on
short_question_to_node, long_question_to_node = short_question_to_node_dict(folder_contributions)

# Create a list of all short questions in order to cluster them
short_questions = []
for node in folder_contributions:
    short_questions.append(node.shortened_question)

# Returns a list where the index is equal to the cluster label for the question at the same index in short_questions
labels = kmeans_cluster(k, short_questions)

# Returns two dictionaies one that links the short question to its cluster and one that links
# a cluster number to a list of questions
cluster_to_question, question_to_cluster = create_cluster_dicts(labels, short_questions)

for cluster in cluster_to_question:
    for question in cluster_to_question[cluster]:
        if question:
            print(short_question_to_node[question[0:100]].raw_question)
    print("\n\n\n\n\n\n\n\n\n\n\n\n\n")

could we leave it as $$\Theta log(n!)$$
or do we need to keep it as something simpler?
Can Ben select specific candies? Say we index the candies 1 to n. Can Ben choose candies 1 and 3, for example, or must he choose two at random?
For this question, is our algorithm expected to always return the same position consistently. Or, can the algorithm output either position?
Can we just prove the worst case for our runtime in 1B, since that's what was done for the Euclid Algorithm?
Can we just say Partition in our algorithm or do we need to describe it again? Thanks
Does the question mean that Ben wins if for a message "j", there are more than n/2 candies with message "j" written on them? Or does it mean that for any two candies "a" and "b", if they have the same message then that is 1 towards our n/2 + 1 threshold?
Can we format the algorithm description in bullet list form? 
When the questions ask for a ____ time algorithm, for example in Q3. "Describe a linear time algorithm (with respect 

In [16]:
from BERTMethods import create_bert_model

bert_models = {}
raw_questions_for_bert = {}

# We want to iterate through and then come up with a BERT model for each cluster
for cluster_number in cluster_to_question:
    # Strip all questions so that we can determine which are empty
    raw_questions_for_bert[cluster_number] = [short_question_to_node[x[0:100]].raw_question for x in cluster_to_question[cluster_number] if x.strip()]
    bert_models[cluster_number] = create_bert_model(raw_questions_for_bert[cluster_number])

# BERT models is now a dict where each value is of the form (bc, doc_vecs)

/Users/maralevy/miniconda3/lib/python3.7/site-packages/bert_serving/client/__init__.py:285: UserWarning: some of your sentences have more tokens than "max_seq_len=100" set on the server, as consequence you may get less-accurate or truncated embeddings.
here is what you can do:
- disable the length-check by create a new "BertClient(check_length=False)" when you do not want to display this warning
- or, start a new server with a larger "max_seq_len"
  '- or, start a new server with a larger "max_seq_len"' % self.length_limit)


In [17]:
from BERTMethods import find_k_similar_sentences
topk_per_question = {}

# topk_per_question becomes a dict of the form question: ranked similarities for all questions in that cluster
for cluster_number in raw_questions_for_bert:
    bert_model = bert_models[cluster_number]
    for question in raw_questions_for_bert[cluster_number]:
        final_scores = find_k_similar_sentences(bert_model[0], bert_model[1], 10, raw_questions_for_bert[cluster_number], question)
        if len(list(final_scores.values())) > 0:
            topk_per_question[question] = final_scores[question]

In [18]:
from BERTMethods import cluster_bert_clusters

initial_bert_clusters = cluster_bert_clusters(topk_per_question, k)

final_bert_clusters = {}
final_bert_node_to_cluster = {}

for cluster in initial_bert_clusters:
    print(cluster)
    if len(initial_bert_clusters[cluster]) > 1:
        print(initial_bert_clusters[cluster])

for cluster in initial_bert_clusters.keys():  
    for question in initial_bert_clusters[cluster]:
        final_bert_clusters.setdefault(cluster, []).append(long_question_to_node[question[0:100]])
        final_bert_node_to_cluster[long_question_to_node[question[0:100]].shortened_question[0:100]] = cluster

# print(initial_bert_clusters)

0
1
3
["Can we just prove the worst case for our runtime in 1B, since that's what was done for the Euclid Algorithm?", 'For example, quickselect has an expected linear runtime but worstcase of n^2.\n\nCan we do something like that for Q3 or do we have to come up with worst case linear', 'If we were to derive an algorithm that executes in better than nlogn\xa0time would we receive\xa0credit?']
4
['Can we just say Partition in our algorithm or do we need to describe it again? Thanks', "Can we directly use the algorithm and runtime from rajiv's lecture by simply referring to it? Do we have to describe the algorithm again?", 'Can we use deterministic selection directly from lecture, or should we describe the algorithm a second time?']
5
6
7
8
9
10
11
12
['For question three, if my list was [1, 2, 3, 100, 101], would calling the algorithm with s=3 return [1,2,3] or [2,3,100]? Basically, does "closest" mean proximity to the median in the sorted list, or difference in value?', 'For question 3

In [19]:
from ManipulationMethods import find_clusters_to_combine

groups = find_clusters_to_combine(all_contributions, final_bert_node_to_cluster, len(list(initial_bert_clusters.keys())), short_question_to_node)

In [20]:
print(groups)

final_clusters = []

for curr_group in groups:
    curr_cluster = []
    for num in curr_group:
        if num in final_bert_clusters:
            for node in final_bert_clusters[num]:
                curr_cluster.append(node)
    if len(curr_cluster) > 1:
        final_clusters.append(curr_cluster)

[[0], [1], [2], [4], [6], [7], [8], [9], [10], [11], [13], [14], [15], [16], [17], [18], [19], [20, 40, 47, 48], [3, 21, 26, 37], [22], [23], [24], [25], [27], [28], [29], [30], [31], [33], [34], [35], [36], [38], [39], [41], [42], [44], [12, 45], [46], [49], [50], [51], [5, 32, 43, 52, 53, 57, 61, 64, 65, 67], [54], [55], [56], [58], [59], [60], [62], [63], [66], [68], [69]]


In [21]:
import csv

with open('output.csv','w') as f:
    writer = csv.writer(f)
    for sublist in final_clusters:
        to_write = []
        for item in sublist:
            to_write.append(item.raw_question)
        writer.writerow(to_write)